<a href="https://colab.research.google.com/github/Aaron-LHR/Aaron-LHR.github.io/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
import numpy as np
import matplotlib
import torch
# matplotlib.use('Qt5Agg')
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch import nn
import torch
from tqdm.auto import tqdm
work_dir = "./"

In [2]:
def load_single_data(file_path):
    b = np.loadtxt(file_path, dtype=np.float32)
    b = np.transpose(b)
    # print(b)
    return b

In [3]:
from google.colab import drive
drive.mount('/content/drive')
work_dir = "/content/drive/MyDrive/"

Mounted at /content/drive


# 可视化

In [6]:
single_trace = load_single_data('/content/drive/MyDrive/mss-missile/data/data-v2/1.txt')

In [ ]:
def visualize_trace(trace, figure_index, x_index=1):
    fig = plt.figure()
    ax = fig.add_subplot(projection='3d')
    # 轨迹
    ax.plot(xs=trace[x_index], ys=trace[x_index + 1], zs=trace[x_index + 2])
    # 起点
    ax.scatter(trace[x_index][:1], trace[x_index + 1][:1], trace[x_index + 2][:1], 'r')
    # 飞行姿态变化
    first_change_time = trace[-2][0]
    first_change_time_index = int(first_change_time // 0.05)
    second_change_time = first_change_time + trace[-2][1]
    second_change_time_index = int(second_change_time // 0.05)
    ax.scatter(trace[x_index][first_change_time_index], trace[x_index + 1][first_change_time_index], trace[x_index + 2][first_change_time_index], 'r')
    ax.scatter(trace[x_index][second_change_time_index], trace[x_index + 1][second_change_time_index], trace[x_index + 2][second_change_time_index], 'r')
    fig.savefig(f"./figure/new_trace_{figure_index}.png", dpi=300)
    fig.show()

In [ ]:
for index in range(1, 21):
    visualize_trace(load_single_data(f'./data/data-v2/{index}.txt'), index)

In [ ]:
def get_end_point(trace):
    print(f"起点经纬高:({', '.join([str(round(x, 2)) for x in trace[-1][:3]])})\t终点经纬高:({', '.join([str(round(x, 2)) for x in trace[-1][3:6]])})")
    print(f"起点坐标系:({', '.join([str(round(x, 2)) for x in trace[1:4, 0]])})\t终点坐标系:({', '.join([str(round(x, 2)) for x in trace[1:4, -1]])})")

get_end_point(single_trace)

起点经纬高:(1.73, 0.5, 2530.28)	终点经纬高:(-2.21, 4.94, 3495.68)
起点坐标系:(0.0, 0.0, 0.0)	终点坐标系:(1117250.0, 973.58, 0.04)


# 训练

In [4]:
seq_length = 10
def generate_train_data(single_data, seq_length = seq_length):
  start_point = single_data[0][1:4]
  start_point_col = np.tile(start_point, (single_data.shape[0] - 1, 1))
  # print(start_point_col.shape)
  end_point = single_data[-1][1:4]
  end_point_col = np.tile(end_point, (single_data.shape[0] - 1, 1))
  # print(end_point_col.shape)
  target = single_data[1:, 1:4]
  # time, sx, sy, sz, x, y, z, ex, ey, ez
  features = np.concatenate((start_point_col, single_data[:-1, :4], end_point_col),axis=1)
  # print(features.shape)
  for i in range(seq_length - 1):
      features = np.insert(features, 1, features[0], axis=0)
  # print(features.shape)
  
  return features, target

generate_train_data(np.loadtxt(work_dir + 'mss-missile/data/data-v2/1.txt', dtype=np.float32))[0][:2]

array([[0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.11725e+06, 9.73583e+02, 3.80787e-02],
       [0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00, 0.00000e+00,
        0.00000e+00, 0.00000e+00, 1.11725e+06, 9.73583e+02, 3.80787e-02]],
      dtype=float32)

In [5]:
root_data_path = work_dir + "mss-missile/data/data-v2/"
data_list = []
label_list = []
lens_list = []
for file_name in os.listdir(root_data_path):
    data_file_path = os.path.join(root_data_path, file_name)
    single_data = np.loadtxt(data_file_path, dtype=np.float32)
    single_train_data, single_label = generate_train_data(single_data)
    print(single_train_data.shape)
    data_list.append(single_train_data)
    label_list.append(single_label)
    lens_list.append(single_train_data.shape[0])
max_seq_len = max([x.shape[0] for x in data_list])

(13468, 10)
(13468, 10)
(13468, 10)
(13727, 10)
(13994, 10)
(13468, 10)
(14549, 10)
(14268, 10)
(14549, 10)
(14268, 10)
(13727, 10)
(13994, 10)
(14268, 10)
(14549, 10)
(13727, 10)
(13994, 10)
(14268, 10)
(14549, 10)
(13994, 10)
(13727, 10)


In [6]:
same_size_data_list = []
same_size_label_list = []
for index, x in enumerate(data_list):
    if x.shape[0] < max_seq_len:
        x = np.concatenate((x, np.zeros((max_seq_len - x.shape[0], 10), np.float32)), axis=0)
    same_size_data_list.append(x)
for index, x in enumerate(label_list):
    if x.shape[0] < max_seq_len:
        x = np.concatenate((x, np.zeros((max_seq_len - x.shape[0], 3), np.float32)), axis=0)
    same_size_label_list.append(x)
train_batch_all = np.stack(same_size_data_list)
label_all = np.stack(same_size_label_list)

In [7]:
train_data_tensor = torch.from_numpy(train_batch_all)
label_tensor = torch.from_numpy(label_all)

In [8]:
class TraceModel(torch.nn.Module):
    def __init__(self, input_size = 10, hidden_size = 20, output_size = 3):
        super().__init__()
        self.batch_first = True
        self.lstm = torch.nn.LSTM(input_size, hidden_size, 2, batch_first=self.batch_first) # (input_size, hidden_size, num_layers)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, input_tensor, seq_lens=[]):
        # input_tensor shape: batch_size*time_step*dim , seq_lens: (batch_size,)  when batch_first = True
        # total_length = input_tensor.size(1) if self.batch_first else input_tensor.size(0)
        # x_packed = pack_padded_sequence(input_tensor, seq_lens, batch_first=self.batch_first, enforce_sorted=False)
        y_lstm, hidden = self.lstm(input_tensor)
        # y_padded, length = pad_packed_sequence(y_lstm, batch_first=self.batch_first, total_length=total_length)
        res = self.linear(y_lstm[:,-1])
        return res

In [9]:
model = TraceModel()
xyz = model(train_data_tensor, lens_list) # (batch, seq_len, output_size)

In [10]:
xyz.shape

torch.Size([20, 3])

In [14]:
device = "cuda"
# train_data_tensor.to(device)
model = TraceModel()
model.to(device)
model.train()
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 建立优化器实例
epochs = 10
for i in tqdm(range(epochs)):
    for seq_start_index in tqdm(range(max_seq_len - seq_length), colour='orange'):
        #清除网络先前的梯度值
        optimizer.zero_grad()
        # #初始化隐藏层数据
        # model.hidden_cell = (torch.zeros(1, 1, model.hidden_layer_size),
        #                       torch.zeros(1, 1, model.hidden_layer_size))
        #实例化模型
        seq = train_data_tensor[:, seq_start_index: seq_start_index + seq_length].to(device)
        y_pred = model(seq).to('cpu')

        #计算损失，反向传播梯度以及更新模型参数
        #训练过程中，正向传播生成网络的输出，计算输出和实际值之间的损失值
        single_loss = loss_function(y_pred, label_tensor[:, seq_start_index + seq_length])
        single_loss.backward()#调用backward()自动生成梯度
        optimizer.step()#使用optimizer.step()执行优化器，把梯度传播回每个网络

        # 查看模型训练的结果
    # if i % 25 == 1:
    print(f'epoch: {i:3} loss: {single_loss.item(): 10.8f}')
print(f'epoch: {i} loss: {single_loss.item(): 10.10f}')
torch.save(model, work_dir + "mss-missile/model/lstm-seq_len_10-hidden_size_20-num_layers_2-linear_1-epoch_10-mse_loss-adam.chpk")

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/14539 [00:00<?, ?it/s]

  0%|          | 0/14539 [00:00<?, ?it/s]

epoch:   1 loss:  47108.14843750


  0%|          | 0/14539 [00:00<?, ?it/s]

...

  0%|          | 0/14539 [00:00<?, ?it/s]

  0%|          | 0/14539 [00:00<?, ?it/s]

  0%|          | 0/14539 [00:00<?, ?it/s]

  0%|          | 0/14539 [00:00<?, ?it/s]

epoch:   9 loss:  924788.0625000000


# 练习lstm

In [8]:
rnn = nn.LSTM(10, 20, 2) # (input_size,hidden_size,num_layers)
input = torch.randn(5, 3, 10) # (seq_len, batch, input_size)
h0 = torch.randn(2, 3, 20) # (num_layers,batch,output_size)
c0 = torch.randn(2, 3, 20) # (num_layers,batch,output_size)
output, (hn, cn) = rnn(input, (h0, c0))

In [ ]:
input.shape

torch.Size([5, 3, 20])

In [ ]:
output.shape #(seq_len, batch, output_size)

In [ ]:
hn.shape # (num_layers, batch, output_size)

In [ ]:
cn.shape # (num_layers, batch, output_size)

In [19]:
len(y_padded[0])

14549

In [16]:
len(y_padded[0][-1])

3

In [17]:
y_padded[0][-1]

tensor([0., 0., 0.], grad_fn=<SelectBackward0>)

In [20]:
len(hidden[0])

2

In [21]:
hidden[0]

tensor([[[-0.0000, -0.7616,  0.0000,  0.0000, -1.0000,  1.0000, -1.0000,
           0.0000, -0.0000, -1.0000,  0.0000,  0.0000, -0.7616,  0.0000,
           0.0000,  0.0000,  0.0000, -0.7616,  0.0000, -0.0000],
         [-0.0000, -0.7616,  0.0000,  0.0000, -1.0000,  1.0000, -1.0000,
           0.0000, -0.0000, -1.0000,  0.0000,  0.0000, -0.7616,  0.0000,
           0.0000,  0.0000,  0.0000, -0.7616,  0.0000, -0.0000],
         [-0.0000, -0.7616,  0.0000,  0.0000, -1.0000,  1.0000, -1.0000,
           0.0000, -0.0000, -1.0000,  0.0000,  0.0000, -0.7616,  0.0000,
           0.0000,  0.0000,  0.0000, -0.7616,  0.0000, -0.0000],
         [-0.0000, -0.7616,  0.0000,  0.0000, -1.0000,  1.0000, -1.0000,
           0.0000, -0.0000, -1.0000,  0.0000,  0.0000, -0.7616,  0.0000,
           0.0000,  0.0000,  0.0000, -0.7616,  0.0000, -0.0000],
         [-0.0000, -0.7616,  0.0000,  0.0000, -1.0000,  1.0000, -1.0000,
           0.0000, -0.0000, -1.0000,  0.0000,  0.0000, -0.7616,  0.0000,
          